In [1]:
import numpy as np
import math
from scipy.stats import norm
import matplotlib.pyplot as plt

In [2]:
def linear_congruential_generator(N):
    """Generates uniform random samples on [0,1]."""
    
    ## Parameters for the Linear Congruential Generator.
    a = 39373
    c = 0
    k = 2**31 - 1

    samples = np.zeros(N)
    xi = 1

    for i in range(N):
        xi = (a * xi + c) % k
        ui = xi / k
        samples[i] = ui
    
    return samples

## Test Script:
# N = 10
# print(linear_congruential_generator(N))

In [3]:
def inverse_normal_approximation(u):
    """Generates normally distributed realizations from uniform random realizations."""
    
    ## Constants for approximations to inverse normal.
    a0, a1, a2, a3 = 2.50662823884, -18.61500062529, 41.39119773534, -25.44106049637
    b0, b1, b2, b3 = -8.47351093090, 23.08336743743, -21.06224101826, 3.13082909833
    c0, c1, c2, c3 = 0.3374754822726147, 0.9761690190917186, 0.1607979714918209, 0.0276438810333863
    c4, c5, c6, c7, c8 = 0.0038405729373609, 0.0003951896511919, 0.0000321767881768, 0.0000002888167364, 0.0000003960315187

    y = u - 0.5
    if abs(y) < 0.42:
        r = y * y
        x = y * (((a3 * r + a2) * r + a1) * r + a0) / ((((b3 * r + b2) * r + b1) * r + b0) * r + 1)
    else:
        r = u
        if y > 0:
            r = 1 - u
        r = math.log(-math.log(r))
        x = c0 + r * (c1 + r * (c2 + r * (c3 + r * (c4 + r * (c5 + r * (c6 + r * (c7 + r * c8)))))))
        if y < 0:
            x = -x

    return x

## Test Script:
# u = 0.8
# print(inverse_normal_approximation(u))
## Results verified here: https://statisticshelper.com/inverse-normal-distribution-calculator/#answer

## Test Script 2:
# N = 1000
# k = 5
# z = np.vectorize(inverse_normal_approximation)(linear_congruential_generator(N*(2**k)))
# plt.hist(z)

In [4]:
def d1(S, K, T, r, q, sigma):
    """Calculates d1 (BSM)."""
    return (math.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))

def d2(S, K, T, r, q, sigma, d1_val=None):
    """Calculates d2 (BSM)."""
    if d1_val is None:
        d1_val = d1(S, K, T, r, q, sigma)
    return d1_val - sigma * math.sqrt(T)

def bs_call(S, K, T, r, q, sigma):
    """Calculate the value for a European call option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    d2_val = d2(S, K, T, r, q, sigma, d1_val)
    return S * math.exp(-q * T) * norm.cdf(d1_val) - K * math.exp(-r * T) * norm.cdf(d2_val)

def bs_put(S, K, T, r, q, sigma):
    """Calculate the value for a European put option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    d2_val = d2(S, K, T, r, q, sigma, d1_val)
    return K * math.exp(-r * T) * norm.cdf(-d2_val) - S * math.exp(-q * T) * norm.cdf(-d1_val)

def bs_call_delta(S, K, T, r, q, sigma):
    """Calculate the delta for a European call option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    return math.exp(-q * T) * norm.cdf(d1_val)

def bs_put_delta(S, K, T, r, q, sigma):
    """Calculate the delta for a European put option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    return -math.exp(-q * T) * norm.cdf(-d1_val)

def bs_gamma(S, K, T, r, q, sigma):
    """Calculate the gamma of a European option (BSM). Note: same for call and put option."""
    d1_val = d1(S, K, T, r, q, sigma)
    return np.exp(-q * T) * norm.pdf(d1_val) / (S * sigma * math.sqrt(T))

def bs_vega(S, K, T, r, q, sigma):
    """Calculate the vega of a European option (BSM). Note: same for call and put option."""
    d1_val = d1(S, K, T, r, q, sigma)
    return S *  math.exp(-q * T) * math.sqrt(T) * norm.pdf(d1_val)

In [5]:
def simulated_spot(S0, T, r, q, sigma, zi):
    """Calculate the simulated spot price S_i at time T."""
    return S0 * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * zi)

def call_payoff(Si, K, T, r):
    """Calculate the discounted payoff of a European call option at time T."""
    return np.exp(-r * T) * np.maximum(Si - K, 0)

def put_payoff(Si, K, T, r):
    """Calculate the discounted payoff of a European put option at time T."""
    return np.exp(-r * T) * np.maximum(K - Si, 0)

def call_delta_estimate(Si, S0, K, T, r):
    """Calculate the delta estimate of the European call option at time T."""
    indicator = (Si > K).astype(int) # Convert boolean array to int array
    return indicator * np.exp(-r * T) * Si / S0

def put_delta_estimate(Si, S0, K, T, r):
    """Calculate the delta estimate of the European put option at time T."""
    indicator = (K > Si).astype(int) * -1 # Convert boolean array to int array
    return indicator * np.exp(-r * T) * Si / S0

def call_vega_estimate(Si, K, T, r, sigma, zi):
    """Calculate the vega estimate of the European call option at time T."""
    indicator = (Si > K).astype(int) # Convert boolean array to int array
    return indicator * Si * np.exp(-r * T) * (-sigma * T + np.sqrt(T) * zi)

def put_vega_estimate(Si, K, T, r, sigma, zi):
    """Calculate the vega estimate of the European put option at time T."""
    indicator = (K > Si).astype(int) * -1 # Convert boolean array to int array
    return indicator * Si * np.exp(-r * T) * (-sigma * T + np.sqrt(T) * zi)

In [6]:
def box_muller(u1,u2):
    u1 = 2 * u1 - 1
    u2 = 2 * u2 - 1
    x = u1**2 + u2**2
    if x > 1:
        return None
    y = np.sqrt(-2*math.log(x)/x)
    z1 = u1 * y
    z2 = u2 * y
    return (z1,z2)

Part4

By Zhuo

In [7]:
#The parameters for the asset
S0 = 50
sigma = 0.3
V0 = 0.09
lam = 4
V_mean = 0.35**2
eta = 0.25
q = 0
rho = -0.15

#The parameters for the put
T = 0.5
K = 50
r = 0.05

#The parameters for the model
m = 175
delta_t = T / m


In [8]:
p_true = bs_put(S0, K, T, r, q, sigma)
print(p_true)

3.5829339156412274


In [28]:
values = []
for k in range(6):
    n = 500 * (2**k)
    N = 2 * 175 * n # number of independent normal rvs needed
    U = linear_congruential_generator(int(1.5*N)) # To generate N independent normal rvs, approximately needs 1.5*N independent uniform rvs
    sum_v = 0
    i = 0
    count = 0
    while count < 175 * n:
        time = 0
        S = S0
        V = V0
        while time < 175:
            u1 = U[2*i]
            u2 = U[2*i+1]
            i += 1
            tup = box_muller(u1,u2)
            if tup != None:
                zi,zj = tup
                count += 1
                S = S * np.exp((r - max(V,0)/2) * delta_t + np.sqrt(max(V,0) * delta_t) * zi)
                V = max(V,0) - lam * (max(V,0) - V_mean) * delta_t + eta * np.sqrt(max(V,0) * delta_t) * (rho * zi + np.sqrt(1 - rho**2) * zj)
                time += 1
        sum_v += put_payoff(S, K, T, r)
    value_hat = sum_v / n
    values.append(value_hat)

In [29]:
values

[3.9588521348855616,
 3.7352167699475767,
 3.961409951594242,
 4.025638237540043,
 4.014398561331073,
 3.9814912578023693]

In [30]:
#find the impied volatility
def bisection(f,x1,x2,tol):
    while max(abs(f(x1)),abs(f(x2)))>tol:
        mid = (x1+x2)/2
        if f(mid)*f(x1) < 0:
            x2 = mid
        else:
            x1 = mid
    return x1

In [31]:
imps = []
for v in values:
    def implied(sigma0):
        return bs_put(S0, K, T, r, q, sigma0) - v
    #print(implied(0.3))
    #print(implied(0.4))
    imp = bisection(implied,0.3,0.5,10**(-6))
    imps.append(imp)

In [32]:
imps

[0.3273270606994628,
 0.3110702037811279,
 0.3275129795074462,
 0.33218212127685537,
 0.3313650608062743,
 0.32897281646728516]